In [1]:
pip install numpy pandas scikit-learn surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=586861 sha256=be8f4f15300aaf315e0e825c878278a2ed74e75ccb9a8065a69b92c06557b4a4
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel




In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zLd1KhGvE86KgMTp7FeFbFlN2jf7q0Bx1K_T8F1IMCMs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud')

bucket = 'autoaimovierecommendationsystem-donotdelete-pr-vjiupdinfceo3p'
object_key = 'dataset.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

movies = pd.read_csv(body)
movies.head(10)


,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811
5,667257,Impossible Things,"Family,Drama",es,"Matilde is a woman who, after the death of her...",14.358,2021-06-17,8.6,255
6,129,Spirited Away,"Animation,Family,Fantasy",ja,"A young girl, Chihiro, becomes trapped in a st...",92.056,2001-07-20,8.5,13093
7,730154,Your Eyes Tell,"Romance,Drama",ja,"A tragic accident lead to Kaori's blindness, b...",51.345,2020-10-23,8.5,339
8,372754,Dou kyu sei – Classmates,"Romance,Animation",ja,"Rihito Sajo, an honor student with a perfect s...",14.285,2016-02-20,8.5,239
9,372058,Your Name.,"Romance,Animation,Drama",ja,High schoolers Mitsuha and Taki are complete s...,158.270,2016-08-26,8.5,8895


In [23]:

tfidf = TfidfVectorizer(stop_words='english')
movies['genre'] = movies['genre'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies['genre'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()


In [24]:

def recommend_movies(genre, num_recommendations=10):
    
    genre_movies = movies[movies['genre'].str.contains(genre, case=False, na=False)]
    
    if genre_movies.empty:
        return pd.DataFrame(columns=['title', 'genre'])  # Return an empty DataFrame if no movies match the genre
    
    
    mean_sim_scores = cosine_sim[genre_movies.index].mean(axis=0)
    
    
    sim_scores = list(enumerate(mean_sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:num_recommendations]
    

    movie_indices = [i[0] for i in sim_scores]
    
    
    recommended_movies = movies.iloc[movie_indices]
    recommended_movies = recommended_movies[recommended_movies['genre'].str.contains(genre, case=False, na=False)]
    
    return recommended_movies[['title', 'genre']]



In [25]:
genre = 'Drama'
recommendations = recommend_movies(genre)
print(recommendations)

                              title  genre
10                     12 Angry Men  Drama
32  One Flew Over the Cuckoo's Nest  Drama
33                       Fight Club  Drama
37                             Hope  Drama
43                 Sunset Boulevard  Drama
46                            Ikiru  Drama
52               American History X  Drama
67               Dead Poets Society  Drama
78                      Better Days  Drama
81                      Tokyo Story  Drama
